## 직종 _ 빈도수 _ 나열 만들기

In [4]:
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import re
import urllib.request
from collections import Counter 
from nltk.tokenize import word_tokenize # 단어단위로 나누는 
from konlpy.tag import Okt
from konlpy.tag import Mecab

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Embedding, Dropout, BatchNormalization, Conv1D, GlobalAveragePooling1D, Bidirectional, LSTM, Flatten, MaxPooling1D, Reshape
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [5]:
import pandas as pd
print("pandas version: ", pd.__version__)
pd.set_option('display.max_row', 500)
pd.set_option('display.max_columns', 100)

df = pd.read_csv('../data/title_for_ml_1.csv')

pandas version:  1.3.4


In [6]:
df.head()

,TITLE,JK_NAME,TITLE_analyze,JK_NAME_analyze
0,월200만원 a 신한카드 설계사및 신규상품 홍보팀원모집 투잡가능,고객상담·인바운드,월 200 만 원 a 신한 카드 설계 사 및 신규 상품 홍보 팀 원 모집 투잡 가능,고객 상담 인 바운드
1,월200만원 a 신한카드 설계사및 신규상품 홍보팀원모집 투잡가능,금융·보험영업,월 200 만 원 a 신한 카드 설계 사 및 신규 상품 홍보 팀 원 모집 투잡 가능,금융 보험 영업
2,월220 준오헤어 강남1호점 데스크 매장관리 직원,반려동물케어,월 220 준오 헤어 강남 1 호 점 데스크 매장 관리 직원,반려 동물 케어
3,월230만원 일8시간 주6일근무 주방직원 구함,패스트푸드,월 230 만 원 일 8시간 주 6일 근무 주방 직원 구함,패스트 푸드
4,월235 경력무관 준오헤어 서울대입구역점 패션스트릿매장 카운터 관리자 매장관리 고속승진,반려동물케어,월 235 경력 무관 준오 헤어 서울대 입구 역 점 패션 스트릿 매장 카운터 관리자...,반려 동물 케어


In [7]:
print(df.shape)

(499999, 4)


In [ ]:
import re

test = []

for i in range(len(df)):
  text = str(df.TITLE_analyze[i])
  text = re.sub("[^0-9a-zA-Z가-힣:& ]", " ", text)
  text = re.sub(" +"," ", text)
  text = re.sub("^ ","", text)
  test.append(text)

df['TEST'] = test

### label 설정
* 직종명 텍스트 -> 정수변환
* 제목 일정 길이 이하 공고 제외
* class별 공고수 10개 이하인 경우 학습 데이터 제외

In [ ]:
df = df [['JK_NAME', 'TEST']].sample(frac=1).reset_index(drop=True)
df = df[df.TEST.str.len() > 5]
df.reset_index(drop=True, inplace=True)

df_idx = df.groupby(['JK_NAME'])['JK_NAME'].count().reset_index(name='count')
df_idx = df_idx[df_idx['count'] > 10].reset_index(drop=True)
df_idx['idx'] = pd.Series(np.arange(0,len(df_idx)))

df = df.merge(df_idx[['JK_NAME', 'idx']], on='JK_NAME')
df = df.sample(frac=1).reset_index(drop=True)
df